# 指数のクエリからの攻撃

In [1]:
#　データベースを構築　今回の場合はsecretベクトルを再現
import numpy as np
import itertools
import random
from operator import itemgetter
import time

def getQuery(database_num,help_vector):
    query_dict = {}
    # step 1
    # すべてのクエリベクトルを生成する
    for i in range(database_num):
        #重複しない組み合わせを生成する
        for v in itertools.combinations(help_vector, i+1):
          v_set = list(v) # サイズ1からｎまでの組み合わせを生成する
          
          # クエリのベクトル初期化
          q = np.zeros(database_num, dtype=int)
          # c = [0,0,0,0,0,0,0,0,0,0]

          for j in range(len(v_set)):
              # 部分集合からベクトルへ　　　
              # まずは1に変化するindexを探す
              index = v_set[j] - 1
              q[index] = 1
              
              # 部分集合とクエリベクトルを保存する　　　dict型を使う
              d_key = " ".join([str(_) for _ in v_set])
              query_dict[d_key] = q

    # # key と クエリを見る
    # for k, v in query_dict.items():
    #     print(k, v)

    return query_dict

# 応答(y1,...,ybn)を取得し     
# クエリをデータベースに聞く　　　今回の場合はsecretベクトルに聞く
def getQueryResponse(query,d):
    arr_d = np.array(d)

    query_response_dict = {}

    # 新しいクエリの結果を保存する
    for k, v in query.items():
        arr_v = np.array(v)

        # クエリと秘密データベースの結果を得る
        result = np.dot(arr_d,arr_v)

        # クエリはkeyとする、計算した結果をvalueにする
        # 例：　'-1 1 1 -1 -1 1 1 1 -1 1': 4, 
        
        # 結果をdictに保存する
        # if result > 0:
        d_key = " ".join([str(_) for _ in v])
        query_response_dict[d_key] = result

    return query_response_dict

# すべての候補データベースを生成する
def gen_candidate_DB(databese_number,help_vector):
    candidate_DB_dict = {}
    for i in range(databese_number):
        for v in itertools.combinations(help_vector, i+1):
          v_set = list(v)
          
          # 候補データベース
          c = np.zeros(database_num, dtype=int)

          for j in range(len(v_set)):
              # 集合からベクトルへ　　　
              # まずは1に変化するindexを探す
              index = v_set[j] - 1
              c[index] = 1
              
              # 部分集合と候補データベースを保存する　　　dict型を使う
              d_key = " ".join([str(_) for _ in v_set])
              candidate_DB_dict[d_key] = c
    return candidate_DB_dict

# |y − A| = 0 となるようなデータベースcを出力すること 
def getDatabaseC(querry_dict,query_response_dict,help_vector,database_num):
    c_result_dict = {}

    # cを作成する: あらゆる組み合わせをデータベースに候補する
    c_databese = gen_candidate_DB(database_num,help_vector)

    for c_key,c_value in c_databese.items():
    
        # 得られたクエリベクトルと結果をを取り出す
        for q_key,q_value in query_response_dict.items():

            # 候補データベースとクエリの内積を計算する
            c = np.array(list(c_value))  # 候補データベース
            q = np.fromstring(q_key, dtype=int, sep=' ')  #クエリベクトル

            # 候補データベースに聞く結果を計算する
            result = np.dot(c,q)

            # 結果をdictに保存する
            if result > 0:
                d_key = " ".join([str(_) for _ in q])
                c_result_dict[d_key] = result

        # 結果dictと元のdictが同じなら、ｃを出力する   
        print("c_result_dict",c_result_dict)
        print("query_response_dict",query_response_dict)

        if c_result_dict == query_response_dict:
            print("cが構築できた",c_value)
            return c_value
            break

# |y − A| <= noise となるようなデータベースcを出力すること 
def getDatabaseCWithNoise_exp(querry_dict,query_response_dict,help_vector,database_num,query_num):
    c_result_dict = {}
    c_dis_array = []

    # cを作成する: あらゆる組み合わせをデータベースに候補する
    c_databese = gen_candidate_DB(database_num,help_vector)

    for c_key,c_value in c_databese.items():
    
        # 得られたクエリベクトルと結果をを取り出す
        for q_key,q_value in query_response_dict.items():

            # 候補データベースとクエリの内積を計算する
            c = np.array(list(c_value))  # 候補データベース
            q = np.fromstring(q_key, dtype=int, sep=' ')  #クエリベクトル

            # 候補データベースに聞く結果を計算する
            result = np.dot(c,q)

            # 結果をdictに保存する
            if np.abs(result - q_value) <= noise:
                d_key = " ".join([str(_) for _ in q])
                c_result_dict[d_key] = result
            else:
                continue

        if len(c_result_dict) == len(query_response_dict):
                print("回目のとき、ｃが構築できた。c=",c_value)            
                return list(c_value)
                break

def getQueryResponseWithNoise_exp(query,d,noise):
    arr_d = np.array(d)

    random_query_list = query
    query_response_dict = {}

    # 新しいクエリの結果を保存する
    for k, v in random_query_list.items():
        query_v = np.array(v) # クエリベクトル

        # クエリと秘密データベースの結果を得る
        result = np.dot(arr_d,query_v)

        # ノイズをランダムに選択する
        randomNoise = random.randint(-noise, noise)
        # print("randomNoise",randomNoise)

        # クエリはkeyとする、計算した結果をvalueにする
        # 例：　'-1 1 1 -1 -1 1 1 1 -1 1': 4, 
        d_key = " ".join([str(_) for _ in query_v])
        query_response_dict[d_key] = result + randomNoise

    # print(query_response_dict)
    return query_response_dict    


 # 構築したｃとｄの差の計算
def compare_c_d(d,c):
        arr_d = np.array(d)
        d_c_dis = np.linalg.norm(arr_d-c, ord=1) # L1ノルムを計算する        
        print("構築したｃとｄの差は",d_c_dis)

In [2]:
before=time.time()
# 攻撃の例
# d = [0,1,0,1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4] # 構築するとき助かるベクトル
# database_num = 4


# d = [0,1,0,1,0,1,0,1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4, 5, 6, 7, 8] # 構築するとき助かるベクトル
# database_num = 8
# noise = 1
# query_num = 2**database_num

# 時間かかる
# d = [0,1,0,1,0,1,0,1,1,0,1,0,1,0,1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11,12,13,14,15] # 構築するとき助かるベクトル
# database_num = 15


# ｃが構築できた。c= [0 1 0 1 0 1 0 1 1 0]
# 構築したｃとｄの差は 2.0
d = [0,1,0,1,0,1,0,1,0,1] # 秘密ベクトル
help_vector = [1, 2, 3, 4, 5, 6, 7, 8,9,10] # 構築するとき助かるベクトル
database_num = 10
noise = 1
query_num = 2**database_num

query = getQuery(database_num,help_vector) # ｂｎ個のランダムクエリベクトルを作成する
query_response_dict = getQueryResponseWithNoise_exp(query,d,noise)

c = getDatabaseCWithNoise_exp(query,query_response_dict,help_vector,database_num,query_num) # もとの秘密ベクトルを推測しよう
compare_c_d(d,c)

回目のとき、ｃが構築できた。c= [0 1 0 1 1 1 0 1 0 0]
構築したｃとｄの差は 2.0


In [3]:
before=time.time()
# 攻撃の例
d = [0,1,0,1] # 秘密ベクトル
help_vector = [1, 2, 3, 4] # 構築するとき助かるベクトル
database_num = 4

# 攻撃の例
# d = [-1,-1,-1,-1,-1,1,1,1,1,1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # 構築するとき助かるベクトル
# database_num = 10
# query_num = 30

querry_dict = getQuery(database_num,help_vector) # ｂｎ個のランダムクエリベクトルを作成する
query_response_dict = getQueryResponse(querry_dict,d) # Aをえる

# # Aを見る
# for k, v in query_response_dict.items():
#     print(k, v)

getDatabaseC(querry_dict,query_response_dict,help_vector,database_num) # もとの秘密ベクトルを推測しよう

after=time.time()
print(str(database_num)+"個のデータベースを構築するときかかった時間は"+str(after-before)+" seconds ")  

c_result_dict {'1 0 0 0': 1, '1 1 0 0': 1, '1 0 1 0': 1, '1 0 0 1': 1, '1 1 1 0': 1, '1 1 0 1': 1, '1 0 1 1': 1, '1 1 1 1': 1}
query_response_dict {'1 0 0 0': 0, '0 1 0 0': 1, '0 0 1 0': 0, '0 0 0 1': 1, '1 1 0 0': 1, '1 0 1 0': 0, '1 0 0 1': 1, '0 1 1 0': 1, '0 1 0 1': 2, '0 0 1 1': 1, '1 1 1 0': 1, '1 1 0 1': 2, '1 0 1 1': 1, '0 1 1 1': 2, '1 1 1 1': 2}
c_result_dict {'1 0 0 0': 1, '1 1 0 0': 1, '1 0 1 0': 1, '1 0 0 1': 1, '1 1 1 0': 1, '1 1 0 1': 1, '1 0 1 1': 1, '1 1 1 1': 1, '0 1 0 0': 1, '0 1 1 0': 1, '0 1 0 1': 1, '0 1 1 1': 1}
query_response_dict {'1 0 0 0': 0, '0 1 0 0': 1, '0 0 1 0': 0, '0 0 0 1': 1, '1 1 0 0': 1, '1 0 1 0': 0, '1 0 0 1': 1, '0 1 1 0': 1, '0 1 0 1': 2, '0 0 1 1': 1, '1 1 1 0': 1, '1 1 0 1': 2, '1 0 1 1': 1, '0 1 1 1': 2, '1 1 1 1': 2}
c_result_dict {'1 0 0 0': 1, '1 1 0 0': 1, '1 0 1 0': 1, '1 0 0 1': 1, '1 1 1 0': 1, '1 1 0 1': 1, '1 0 1 1': 1, '1 1 1 1': 1, '0 1 0 0': 1, '0 1 1 0': 1, '0 1 0 1': 1, '0 1 1 1': 1, '0 0 1 0': 1, '0 0 1 1': 1}
query_response_di

# ランダムのクエリからの攻撃

# ノイズを入れないバージョン

In [6]:
#　データベースを構築　今回の場合はsecretベクトルを再現
import numpy as np
import itertools
import random
from operator import itemgetter
import time

# step 1
# bn個のランダムベクトルv ∈ {−1, 1}**nを選択し
def getRandomQuery(query_number,databese_number):
    random_query_list = {}

    for i in range(query_number):
        original_query_vector = np.full(databese_number, -1, dtype=int) # [-1....] というベクトルを生成する
        # 何個の成分を変わるか設定
        randomNuｍ = random.randint(0, databese_number)
        randomIndexList = [random.randint(0, databese_number) for i in range(randomNuｍ)] # ランダムのインデックス（部分集合）  [11, 9, 4, 10] 

        # randomNuｍ=0だと
        if randomNuｍ == 0:
            d_key=randomNuｍ
            random_query_list[d_key] = original_query_vector  # [-1,-1,-1...]のクエリを出力する
        else:
            for j in range(randomNuｍ):
                # 集合からベクトルへ　　　
                # まずは1に変化するindexを探す
                index = randomIndexList[j] - 1 #部分集合[1,2]
                # 探したら1に変化するindexを探す
                original_query_vector[index] = 1 #部分集合[1,2]だと、1,2番目の成分を１に変わる、[1,1,-1,-1,-1...]
                
                # クエリからクエリベクトルへ　　　dict型を使う
                # 部分集合とクエリベクトルを保存する　　　dict型を使う
                d_key = " ".join([str(_) for _ in randomIndexList])
                random_query_list[d_key] = original_query_vector  # ランダムクエリ辞書を出力 

    return random_query_list

# 応答(y1,...,ybn)を取得し     
# クエリをデータベースに聞く　　　今回の場合はsecretベクトルに聞く
def getQueryResponse(query,d):
    arr_d = np.array(d)

    random_query_list = query
    query_response_dict = {}

    # 新しいクエリの結果を保存する
    for k, v in random_query_list.items():
        arr_v = np.array(v)

        # クエリと秘密データベースの結果を得る
        result = np.dot(arr_d,arr_v)

        # クエリはkeyとする、計算した結果をvalueにする
        # 例：　'-1 1 1 -1 -1 1 1 1 -1 1': 4, 
        d_key = " ".join([str(_) for _ in v])
        query_response_dict[d_key] = result

    return query_response_dict

# すべての候補データベースを生成する
def gen_candidate_DB(databese_number,help_vector):
    candidate_DB_dict = {}
    for i in range(databese_number):
        for v in itertools.combinations(help_vector, i+1):
          v_set = list(v)
          
          # 候補データベース
          c = np.full(databese_number, -1, dtype=int) # [-1....] というベクトルを生成する c = [-1,-1,-1,-1,-1,-1,....]
          # print("len(c)",len(c))

          for j in range(len(v_set)):
              # 集合からベクトルへ　　　
              # まずは1に変化するindexを探す
              index = v_set[j] - 1
              c[index] = 1
              
              # 部分集合と候補データベースを保存する　　　dict型を使う
              d_key = " ".join([str(_) for _ in v_set])
              candidate_DB_dict[d_key] = c
    return candidate_DB_dict

# |y − A| = 0 となるようなデータベースcを出力すること 
def getDatabaseC(random_querry_dict,query_response_dict,help_vector,database_num):
    query_r_c_response_dict = {}
    query_l_c_response_dict = {}

    # cを作成する: あらゆる組み合わせをデータベースに候補する
    c_databese = gen_candidate_DB(database_num,help_vector)
    c_databese = itemgetter(*c_databese.keys())(c_databese)
    c_databese = np.array(c_databese)

    set_size = c_databese.shape[0]
    i_size = int(set_size/2) + 1

    for i in range(i_size):
        # 中間から分かれる、毎回2個の候補データベースを処理する
        right_c_index = i_size+i
        left_c_index = i_size-i-1
        r_c = c_databese[right_c_index-1:right_c_index]
        l_c = c_databese[left_c_index-1:left_c_index]

        for k, v in random_querry_dict.items():
            arr_v = np.array(v)

            # (A)の結果を得る
            # クエリと秘密データベースの結果を得る
            r_result = np.dot(r_c,arr_v) # <(1,1,1,1),(-1,-1,1,1)>
            l_result = np.dot(l_c,arr_v) # <(1,1,1,1),(-1,-1,1,1)>
            
            try:
                query_r_c_response_dict[k] = r_result[0]
                query_l_c_response_dict[k] = l_result[0]
                if r_result[0] != query_response_dict[k]:
                    continue
            except:
                pass              

        if query_r_c_response_dict == query_response_dict:
            print("%s回目のとき、ｃが構築できた。c="%i,r_c)
            break

        if  query_l_c_response_dict == query_response_dict:
            print("%s回目のとき、ｃが構築できた。c="%i,l_c)       
            break

# 実行

In [7]:
# 5671回目のとき、ｃが構築できた。c= [[-1  1 -1  1 -1  1 -1  1 -1  1  1 -1  1  1 -1]]
# 15個のデータベースを構築するときかかった時間は1.237248182296753 seconds 
# d = [-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,1,1,-1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11,12,13,14,15] # 構築するとき助かるベクトル
# database_num = 15

# 12326回目のとき、ｃが構築できた。c= [[-1 -1 -1  1 -1  1  1  1 -1  1  1  1  1 -1  1]]
# 15個のデータベースを構築するときかかった時間は1.908970594406128 seconds 
# d = [-1,-1,-1,1,-1,1,1,1,-1,1,1,1,1,-1,1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] # 構築するとき助かるベクトル
# database_num = 15

# 242970回目のとき、ｃが構築できた。c= [[-1  1 -1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1  1 -1 -1 -1  1]]
# 20個のデータベースを構築するときかかった時間は40.8 seconds 
# d = [-1,1,-1,1,1,1,-1,-1,-1,1,-1,1,-1,1,1,1,-1,-1,-1,1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] # 構築するとき助かるベクトル
# database_num = 20


before=time.time()

# 攻撃の例
d = [-1,-1,-1,-1,-1,1,1,1,1,1] # 秘密ベクトル
help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # 構築するとき助かるベクトル
database_num = 10
query_num = 30

random_querry_dict = getRandomQuery(query_num,database_num) # ｂｎ個のランダムクエリベクトルを作成する
query_response_dict = getQueryResponse(random_querry_dict,d) # Aをえる

getDatabaseC(random_querry_dict,query_response_dict,help_vector,database_num) # もとの秘密ベクトルを推測しよう

after=time.time()
print(str(database_num)+"個のデータベースを構築するときかかった時間は"+str(after-before)+" seconds ")  

10個のデータベースを構築するときかかった時間は0.07409858703613281 seconds 


In [8]:
def getDatabaseD(query_response_dict,help_vector,databese_num):
    before=time.time()

    query_r_c_response_dict = {}
    query_l_c_response_dict = {}
    
    query_response_array = itemgetter(*query_response_dict.keys())(query_response_dict) # 結果リストを得る
    query_response_array = np.array(query_response_array)

    # cを作成する
    # あらゆる組み合わせをデータベースに候補する
    c_databese = gen_candidate_DB(databese_num,help_vector)
    c_databese = itemgetter(*c_databese.keys())(c_databese)
    c_databese = np.array(c_databese)

    set_size = c_databese.shape[0]
    i_size = int(set_size/2) + 1
    print("c_databese",c_databese.shape)

    for i in range(i_size):
        right_c_index = i_size+i
        left_c_index = i_size-i-1
        r_c = c_databese[right_c_index-1:right_c_index]
        l_c = c_databese[left_c_index-1:left_c_index]

        for k, v in query_response_dict.items():
            arr_list = list(map(int, k.split(" ")))
            arr_v = np.array(arr_list)

            # クエリと候補データベースの結果を得る
            # bn x nの行列と候補データベースの内積を計算する
            r_result = np.dot(r_c,arr_v.T) # ([1,1,1,1],[-1,-1,1,1])
            l_result = np.dot(l_c,arr_v.T) # ([1,1,1,1],[-1,-1,1,1])

            # 新しいクエリの結果を保存する
            try:
                query_r_c_response_dict[k] = r_result[0]
                query_l_c_response_dict[k] = l_result[0]
                if r_result[0] != query_response_dict[k]:
                    continue
            except:
                pass              

        if query_r_c_response_dict == query_response_dict:
            print("%s回目のとき、ｃが構築できた。c="%i,r_c)
            break

        if  query_l_c_response_dict == query_response_dict:
            print("%s回目のとき、ｃが構築できた。c="%i,l_c)       
            break


In [9]:
help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # 構築するとき助かるベクトル
query_num = 10
databese_num = 10

query_response_dict = {'-1 1 1 -1 -1 1 1 1 -1 1': 4, 
                       '-1 1 1 1 1 1 1 -1 -1 1': 2,
                       '-1 1 1 1 1 1 1 1 -1 -1': 2,
                       '1 1 1 -1 1 -1 1 1 -1 -1': -4,
                       '1 -1 1 -1 1 -1 -1 1 -1 1': -2,
                       '-1 -1 1 -1 1 1 1 -1 -1 1': -2,
                       '-1 -1 -1 -1 -1 -1 -1 -1 -1 -1': 0,
                       '-1 -1 -1 -1 -1 -1 -1 1 1 1': 2,
                       '-1 -1 1 1 1 -1 1 1 -1 1': 0}

getDatabaseD(query_response_dict,help_vector,databese_num)

c_databese (1023, 10)
49回目のとき、ｃが構築できた。c= [[-1  1 -1  1 -1  1 -1  1 -1  1]]


In [ ]:
tuple = (1, 2, 3)
array2 = np.array(tuple)
print(array2)

[1 2 3]


In [10]:
import pandas as pd

query_response_dict = {'-1 1 1 -1 -1 1 1 1 -1 1': 4, 
                       '-1 1 1 1 1 1 1 -1 -1 1': 2,
                       '-1 1 1 1 1 1 1 1 -1 -1': 2,
                       '1 1 1 -1 1 -1 1 1 -1 -1': -4,
                       '1 -1 1 -1 1 -1 -1 1 -1 1': -2,
                       '-1 -1 1 -1 1 1 1 -1 -1 1': -2,
                       '-1 -1 -1 -1 -1 -1 -1 -1 -1 -1': 0,
                       '-1 -1 -1 -1 -1 -1 -1 1 1 1': 2,
                       '-1 -1 1 1 1 -1 1 1 -1 1': 0}

result = list(zip(query_response_dict.keys(),query_response_dict.values()))
query_response_df = pd.DataFrame(result, columns=["クエリベクトル","結果"])
query_response_df

,クエリベクトル,結果
0,-1 1 1 -1 -1 1 1 1 -1 1,4
1,-1 1 1 1 1 1 1 -1 -1 1,2
2,-1 1 1 1 1 1 1 1 -1 -1,2
3,1 1 1 -1 1 -1 1 1 -1 -1,-4
4,1 -1 1 -1 1 -1 -1 1 -1 1,-2
5,-1 -1 1 -1 1 1 1 -1 -1 1,-2
6,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1,0
7,-1 -1 -1 -1 -1 -1 -1 1 1 1,2
8,-1 -1 1 1 1 -1 1 1 -1 1,0


# ノイズを加える

In [11]:
# 改善
def getDatabaseCwithNoise(query_response_dict,help_vector,databese_num,query_num,noise):
    before=time.time()

    query_r_c_response_dict = {}
    query_l_c_response_dict = {}
    
    query_response_array = itemgetter(*query_response_dict.keys())(query_response_dict) # 結果リストを得る
    query_response_array = np.array(query_response_array)

    # cを作成する
    # あらゆる組み合わせをデータベースに候補する
    c_databese = gen_candidate_DB(databese_num,help_vector)
    c_databese = itemgetter(*c_databese.keys())(c_databese)
    c_databese = np.array(c_databese)

    set_size = c_databese.shape[0]
    i_size = int(set_size/2) + 1
    print("c_databese",c_databese.shape)

    for i in range(i_size):
        right_c_index = i_size+i
        left_c_index = i_size-i-1
        r_c = c_databese[right_c_index-1:right_c_index]
        l_c = c_databese[left_c_index-1:left_c_index]

        for k, v in query_response_dict.items():
            arr_list = list(map(int, k.split(" ")))
            arr_v = np.array(arr_list)

            # クエリと候補データベースの結果を得る
            # bn x nの行列と候補データベースの内積を計算する
            r_result = np.dot(r_c,arr_v.T) # ([1,1,1,1],[-1,-1,1,1])
            l_result = np.dot(l_c,arr_v.T) # ([1,1,1,1],[-1,-1,1,1])

            # 新しいクエリの結果を保存する
            try:
                # root noise 結果が |yi − (Ac)i| ≤ αのみ保存　ノイズを加えたらとの差がノイズより大きくなると排除する
                if np.abs(r_result[0] - v) <= noise:
                    query_r_c_response_dict[k] = r_result[0]
                    query_l_c_response_dict[k] = l_result[0] 
                else:
                    continue

            except:
                pass
        
        if len(query_r_c_response_dict) == len(query_response_dict):
            print("%s回目のとき、ｃが構築できた。c="%i,r_c)
            return list(r_c)
            break
        
        if len(query_l_c_response_dict) == len(query_response_dict):  
            print("%s回目のとき、ｃが構築できた。c="%i,r_c)
            return list(l_c)
            break      
            

def getQueryResponseWithNoise(query,d,noise):
    arr_d = np.array(d)

    random_query_list = query
    query_response_dict = {}

    # 新しいクエリの結果を保存する
    for k, v in random_query_list.items():
        query_v = np.array(v) # クエリベクトル

        # クエリと秘密データベースの結果を得る
        result = np.dot(arr_d,query_v)

        # ノイズをランダムに選択する
        randomNoise = random.randint(-noise, noise)
        # print("randomNoise",randomNoise)

        # クエリはkeyとする、計算した結果をvalueにする
        # 例：　'-1 1 1 -1 -1 1 1 1 -1 1': 4, 
        d_key = " ".join([str(_) for _ in query_v])
        query_response_dict[d_key] = result + randomNoise

    # print(query_response_dict)
    return query_response_dict

 # 構築したｃとｄの差の計算
def compare_c_d(d,c):
        arr_d = np.array(d)
        d_c_dis = np.linalg.norm(arr_d-c, ord=1) # L1ノルムを計算する
        
        print("構築したｃとｄの差は",d_c_dis)


# 攻撃の例

In [ ]:
# c_databese (1023, 10)
# 25回目のとき、ｃが構築できた。c= [[-1  1  1 -1 -1  1  1  1 -1 -1]]
# 構築したｃとｄの差は 2.0
# 10個のデータベースを構築するときかかった時間は0.028502702713012695 seconds 
# d = [-1,1,1,-1,-1,1,-1,1,-1,1] # 秘密ベクトル
# help_vector = [1, 2, 3, 4, 5, 6, 7, 8,9,10] # 構築するとき助かるベクトル
# query_num = 30
# databese_num = 10
# noise = 2

# 242970回目のとき、ｃが構築できた。c= [[-1  1 -1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1  1 -1 -1 -1  1]]
# 20個のデータベースを構築するときかかった時間は40.8 seconds 
d = [-1,1,-1,1,1,1,-1,-1,-1,1,-1,1,-1,1,1,1,-1,-1,-1,1] # 秘密ベクトル
print("len(d)",len(d))
help_vector = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] # 構築するとき助かるベクトル
database_num = 20
query_num = 30
noise = 1

before=time.time()
query = getRandomQuery(query_num,database_num) # ｂｎ個のランダムクエリベクトルを作成する
query_response_dict = getQueryResponseWithNoise(query,d,noise)
print(database_num)
c = getDatabaseCwithNoise(query_response_dict,help_vector,databese_num,query_num,noise)
compare_c_d(d,c)

after=time.time()
print(str(database_num)+"個のデータベースを構築するときかかった時間は"+str(after-before)+" seconds ")  
